In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import load_dataset

ds = load_dataset("youssefedweqd/Diabetic_Retinopathy_Detection")

README.md:   0%|          | 0.00/727 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/328M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/328M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/319M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/108M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/271M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25290 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2810 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7026 [00:00<?, ? examples/s]

In [4]:
import tensorflow as tf

def preprocess_data(example):
    # Normalize the image
    image = example['image'].convert('RGB')  # Ensure the image is RGB
    image = tf.image.resize(image, (224, 224))  # Resize the image
    image = tf.cast(image, tf.float32) / 255.0  # Normalize pixel values

    # Ensure the label is an integer
    label = tf.cast(example['label'], tf.int32)

    return image, label


In [5]:
import tensorflow as tf

# Convert the Hugging Face dataset to a TensorFlow dataset
tf_train_dataset = ds["train"].to_tf_dataset(
    columns=["image"],  # Columns of the dataset to use as features
    label_cols="label",  # Column to use as the label
    shuffle=True,  # Shuffle the data
    batch_size=32,  # Batch size
    collate_fn=lambda batch: {
        "image": tf.image.resize(tf.stack([item['image'] for item in batch]), (224, 224)) / 255.0,  # Process images
        "label": tf.convert_to_tensor([item['label'] for item in batch], dtype=tf.int32)  # Process labels
    }
)

tf_val_dataset = ds["validation"].to_tf_dataset(
    columns=["image"],
    label_cols="label",
    shuffle=False,
    batch_size=32,
    collate_fn=lambda batch: {
        "image": tf.image.resize(tf.stack([item['image'] for item in batch]), (224, 224)) / 255.0,
        "label": tf.convert_to_tensor([item['label'] for item in batch], dtype=tf.int32)
    }
)


/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


#ResNet50 Model

In [6]:
import numpy as np
import tensorflow as tf
from PIL import Image
import requests
from io import BytesIO

# Load the pre-trained ResNet50 model + custom classification layers
base_model = tf.keras.applications.ResNet50(
    include_top=False,  # Exclude the top layers, so we can add custom ones
    weights='imagenet',  # Load pre-trained weights from ImageNet
    input_shape=(224, 224, 3)  # Input shape expected by ResNet50
)

# Freeze the ResNet50 layers to keep the pre-trained weights
base_model.trainable = False

# Create the final model by adding custom layers on top of ResNet50
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # Assuming 5 classes, adjust as needed
])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
# Convert the Hugging Face dataset to a TensorFlow dataset
tf_train_dataset = ds["train"].to_tf_dataset(
    columns=["image"],  # Columns of the dataset to use as features
    label_cols="label",  # Column to use as the label
    shuffle=True,  # Shuffle the data
    batch_size=32,  # Batch size
    collate_fn=lambda batch: {
        "image": tf.image.resize(tf.stack([tf.image.resize(item['image'], (224, 224)) for item in batch]), (224, 224)) / 255.0,  # Resize and normalize images
        "label": tf.convert_to_tensor([item['label'] for item in batch], dtype=tf.int32)  # Process labels
    }
)

tf_val_dataset = ds["validation"].to_tf_dataset(
    columns=["image"],
    label_cols="label",
    shuffle=False,
    batch_size=32,
    collate_fn=lambda batch: {
        "image": tf.image.resize(tf.stack([tf.image.resize(item['image'], (224, 224)) for item in batch]), (224, 224)) / 255.0,
        "label": tf.convert_to_tensor([item['label'] for item in batch], dtype=tf.int32)
    }
)


In [9]:

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [10]:

# Train the model
history = model.fit(tf_train_dataset, epochs=1, validation_data=tf_val_dataset)


791/791 ━━━━━━━━━━━━━━━━━━━━ 4960s 6s/step - accuracy: 0.7275 - loss: 0.8941 - val_accuracy: 0.7349 - val_loss: 0.8762


In [11]:

# Evaluate the model
test_loss, test_acc = model.evaluate(tf_val_dataset)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

88/88 ━━━━━━━━━━━━━━━━━━━━ 497s 6s/step - accuracy: 0.9567 - loss: 0.3849
Test Accuracy: 73.49%


In [13]:
import numpy as np
import tensorflow as tf
from PIL import Image
import requests
from io import BytesIO

# Function to load and preprocess an image
def preprocess_image(image_path):
    # Load the image from file or URL
    if image_path.startswith('http'):
        response = requests.get(image_path)
        img = Image.open(BytesIO(response.content))
    else:
        img = Image.open(image_path)

    # Convert the image to RGB, resize to (224, 224), and normalize
    img = img.convert('RGB')
    img = tf.image.resize(np.array(img), (224, 224))  # Resize image to match the model input
    img = img / 255.0  # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension (1 sample)

    return img

# Function to predict diabetic retinopathy or not
def predict_diabetes(image_path):
    # Preprocess the image
    img = preprocess_image(image_path)

    # Predict using the trained model
    predictions = model.predict(img)

    # Get the class with the highest probability
    predicted_class = np.argmax(predictions)

    # Define class labels (Adjust according to your dataset labels)
    labels = ['Class 0: No Diabetic Retinopathy',
              'Class 1: Mild Diabetic Retinopathy',
              'Class 2: Moderate Diabetic Retinopathy',
              'Class 3: Severe Diabetic Retinopathy',
              'Class 4: Proliferative Diabetic Retinopathy']

    # Output the result
    print(f"Predicted Class: {labels[predicted_class]}")
    return predicted_class, predictions[0][predicted_class]

# Example usage:
image_path = r"/content/drive/MyDrive/dia.jpg" # Path to your image or URL
predicted_class, confidence = predict_diabetes(image_path)
print(f"Confidence: {confidence * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted Class: Class 0: No Diabetic Retinopathy
Confidence: 73.26%
